# Phase 1: Synthetic Data Generation

In [2]:
import pandas as pd
import numpy as np
from sklearn.utils import resample

# Load original data
df_raw = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# Upsample to 100,000 records
df_upsampled = resample(df_raw, replace=True, n_samples=100000, random_state=42)
df_upsampled = df_upsampled.reset_index(drop=True)
df_upsampled['customerID'] = [f"CUST-{i:06d}" for i in range(100000)]

# Randomly assign Treatment (1) or Control (0)
np.random.seed(42)
df_upsampled['is_treatment'] = np.random.binomial(1, 0.5, size=100000)

# Define Causal Logic for stay/churn
def simulate_outcome(row):
    churn_prob = 0.2 if row['Churn'] == 'No' else 0.7

    # Logic for 'Persuadables'
    if row['InternetService'] == 'Fiber optic' and row['tenure'] < 24:
        churn_prob = 0.3 if row['is_treatment'] == 1 else 0.8

    # Logic for 'Sure Things'
    if row['tenure'] > 60:
        churn_prob = 0.05

    return np.random.binomial(1, 1 - np.clip(churn_prob, 0, 1))

df_upsampled['stayed'] = df_upsampled.apply(simulate_outcome, axis=1)

# Introduce data messiness
df_upsampled['TotalCharges'] = pd.to_numeric(df_upsampled['TotalCharges'], errors='coerce')
mask = np.random.choice([True, False], size=100000, p=[0.02, 0.98])
df_upsampled.loc[mask, 'TotalCharges'] = np.nan

# Save dataset
df_upsampled.to_csv('telco_uplift_dataset.csv', index=False)

In [4]:
import sqlite3
import pandas as pd

# 1. Load your synthetic dataset
df = pd.read_csv('telco_uplift_dataset.csv')

# 2. Create a connection to a SQLite database file
# This creates 'telco.db' in your Colab file sidebar
conn = sqlite3.connect('telco.db')

# 3. Write to SQL
# SQLite handles the schema automatically
df.to_sql('stg_telco_uplift', conn, if_exists='replace', index=False)

print("Data successfully stored in SQLite (telco.db).")

Data successfully stored in SQLite (telco.db).


### Phase 1.1 (the SQL ATE calculation)

In [5]:
query = """
WITH base_stats AS (
    SELECT
        InternetService,
        CASE
            WHEN tenure < 24 THEN 'New (<24m)'
            ELSE 'Loyal (>24m)'
        END AS tenure_group,
        is_treatment,
        AVG(stayed) AS avg_stay_rate
    FROM stg_telco_uplift
    GROUP BY 1, 2, 3
)
SELECT
    t.InternetService,
    t.tenure_group,
    t.avg_stay_rate AS treatment_stay_rate,
    c.avg_stay_rate AS control_stay_rate,
    (t.avg_stay_rate - c.avg_stay_rate) AS incremental_lift
FROM base_stats t
JOIN base_stats c ON t.InternetService = c.InternetService
    AND t.tenure_group = c.tenure_group
WHERE t.is_treatment = 1 AND t.is_treatment != c.is_treatment
ORDER BY incremental_lift DESC;
"""

# Execute and view results
lift_df = pd.read_sql(query, conn)
print(lift_df)

  InternetService  tenure_group  treatment_stay_rate  control_stay_rate  \
0     Fiber optic    New (<24m)             0.697633           0.209008   
1     Fiber optic  Loyal (>24m)             0.751453           0.749293   
2             DSL  Loyal (>24m)             0.828721           0.827961   
3              No    New (<24m)             0.728441           0.732902   
4             DSL    New (<24m)             0.629669           0.635786   
5              No  Loyal (>24m)             0.837638           0.846979   

   incremental_lift  
0          0.488625  
1          0.002160  
2          0.000760  
3         -0.004461  
4         -0.006117  
5         -0.009341  


### Phase 1.2 SQL Behavioral Segmentation (RFM)

In [9]:
query = """
-- Creating an Engagement Score (Frequency) and Value Tier (Monetary)
WITH customer_engagement AS (
    SELECT
        customerID,
        tenure, -- Proxy for Recency (Loyalty)
        MonthlyCharges, -- Monetary
        (CASE WHEN PhoneService = 'Yes' THEN 1 ELSE 0 END +
         CASE WHEN MultipleLines = 'Yes' THEN 1 ELSE 0 END +
         CASE WHEN OnlineSecurity = 'Yes' THEN 1 ELSE 0 END +
         CASE WHEN OnlineBackup = 'Yes' THEN 1 ELSE 0 END +
         CASE WHEN DeviceProtection = 'Yes' THEN 1 ELSE 0 END +
         CASE WHEN TechSupport = 'Yes' THEN 1 ELSE 0 END +
         CASE WHEN StreamingTV = 'Yes' THEN 1 ELSE 0 END +
         CASE WHEN StreamingMovies = 'Yes' THEN 1 ELSE 0 END) AS service_count -- Frequency
    FROM stg_telco_uplift
)
SELECT
    *,
    CASE
        WHEN service_count >= 5 AND MonthlyCharges > 80 THEN 'High-Value Power User'
        WHEN service_count >= 3 THEN 'Mid-Tier Engaged'
        ELSE 'Low-Engagement/Basic'
    END AS customer_segment
FROM customer_engagement;
"""

# Phase 2: Data Cleaning & Feature Engineering

In [10]:
# 1. Load data from the CSV we generated
df = pd.read_csv('telco_uplift_dataset.csv')

# 2. Fix the 'TotalCharges' data type (it often reads as object/string)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# 3. Strategic Imputation: Fill NaNs using the business logic (Tenure * Monthly)
# This shows you understand the relationship between features
df['TotalCharges'] = df['TotalCharges'].fillna(df['MonthlyCharges'] * df['tenure'])

# 4. Feature Engineering: Efficiency Metrics
# Creating a 'Charge Velocity' feature to see how fast they are spending
df['charge_velocity'] = df['TotalCharges'] / (df['tenure'] + 1)

# 5. Handle Categorical Encoding
# Convert 'Yes'/'No' to 1/0 for binary columns
binary_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
for col in binary_cols:
    df[col] = df[col].apply(lambda x: 1 if x == 'Yes' else 0)

# 6. One-Hot Encoding for multi-category columns (Contract, PaymentMethod, etc.)
# We drop 'customerID' because it has no predictive power
# We drop 'Churn' because 'stayed' is our new target for Uplift
df_model_ready = pd.get_dummies(
    df.drop(columns=['customerID', 'Churn']),
    drop_first=True
)

print(f"Phase 2 Complete. Feature count increased from {df.shape[1]} to {df_model_ready.shape[1]}.")

Phase 2 Complete. Feature count increased from 24 to 33.


#### Phase 2.1: Analysis for Strategic Recommendation

In [11]:
# Identify which feature correlates most with 'stayed' for new customers (tenure < 12)
new_customers = df_model_ready[df_model_ready['tenure'] < 12]
correlations = new_customers.corr()['stayed'].sort_values(ascending=False)

# Look for 'OnlineSecurity' or 'TechSupport'
print("Top features driving retention for new users:")
print(correlations.head(5))

Top features driving retention for new users:
stayed                                 1.000000
is_treatment                           0.201231
StreamingMovies_No internet service    0.162728
OnlineSecurity_No internet service     0.162728
InternetService_No                     0.162728
Name: stayed, dtype: float64


# Phase 3: Causal Inference & Uplift Modeling (S-Learner)

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# 1. Separate features and target
X = df_model_ready.drop(columns=['stayed'])
y = df_model_ready['stayed']

# 2. Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Initialize and train the Random Forest model
model = RandomForestClassifier(n_estimators=100, max_depth=10, n_jobs=-1, random_state=42)
model.fit(X_train, y_train)

# 4. Calculate Uplift Score: Probability(Stay | Treated) - Probability(Stay | Control)
X_test_treated = X_test.copy()
X_test_treated['is_treatment'] = 1
prob_treated = model.predict_proba(X_test_treated)[:, 1]

X_test_control = X_test.copy()
X_test_control['is_treatment'] = 0
prob_control = model.predict_proba(X_test_control)[:, 1]

# 5. Store the scores
X_test['uplift_score'] = prob_treated - prob_control

# 6. Segment customers based on uplift score quartiles
X_test['segment'] = pd.qcut(X_test['uplift_score'], 4, labels=['Lost Cause', 'Neutral', 'Responders', 'Persuadables'])

print(X_test[['uplift_score', 'segment']].head())

       uplift_score       segment
75721     -0.005343    Lost Cause
80184     -0.006006    Lost Cause
19864     -0.000387       Neutral
76699      0.076732  Persuadables
92991      0.287976  Persuadables


### Export the "Master Tableau File"

In [13]:
# 1. Join uplift scores and segments back to the test features
tableau_df = X_test.copy()
tableau_df['actual_stayed'] = y_test

# 2. Bring back original columns (MonthlyCharges, etc.) for filtering if they were dropped
# Reset index ensures we align the rows correctly
original_data = df.iloc[X_test.index]
tableau_df['InternetService'] = original_data['InternetService'].values
tableau_df['MonthlyCharges'] = original_data['MonthlyCharges'].values

# 3. Export to CSV
tableau_df.to_csv('Tableau_Final_Project.csv', index=False)
print("Tableau file exported. Download 'Tableau_Final_Project.csv' from the file sidebar.")

Tableau file exported. Download 'Tableau_Final_Project.csv' from the file sidebar.
